<div style="text-align: center; background-color: #559cff; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  📌  Trực Quan Hóa Dữ Liệu - 21_21 📌
  
   Lab 1
</div>

# Preprocess

## 1. Load data and take  a quick look

In [1]:
import numpy as np
import pandas as pd

### 1.1 Đọc dữ liệu

In [5]:

chunk_size = 100000  
chunks = pd.read_csv('data/US_Accidents_March23.csv', chunksize=chunk_size)

# List to store individual chunks
chunk_list = []

# Iterate over chunks and append to the list
for chunk in chunks:
    chunk_list.append(chunk)

# Concatenate all chunks into one DataFrame
df = pd.concat(chunk_list, ignore_index=True)



### 1.2 Kích thước dữ liệu

In [6]:
df.shape

(7728394, 46)

### 1.3 Giải thích ý nghĩa các cột

In [3]:
df.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,...,False,False,False,False,True,False,Day,Day,Day,Day


- **ID**: ID của vụ tai nạn
- **Source**: Nguồn dữ liệu thô 
- **Severity**: Mức độ nghiêm trọng của vụ tai nạn, có giá trị từ 1 -> 4, biểu thị mức độ ảnh hưởng tăng dần lên hoạt động giao thông
- **Start_Time**: Thời gian bắt đầu vụ tai nạn theo giờ địa phương
- **End_Time**: Thời gian kết thúc vụ tai nạn, chỉ thời gian ảnh hưởng đến giao thông chấm dứt theo giờ địa phương
- **Start_Lat**: Vĩ độ của địa điểm xảy ra tai nạn
- **Start_Lng**: Kinh độ của địa điểm xảy ra tai nạn
- **End_Lat**: Vĩ độ của địa điểm kết thúc
- **End_Lng**: Kinh độ của địa điểm kết thúc
- **Distance(mi)**: Chiều dài đoạn đường bị ảnh hưởng bởi vụ tai nạn, tính bằng Miles
- **Description**: Mô tả về vụ tai nạn
- **Street**: Tên đường trong trường địa chỉ
- **City**: Tên thành phố trong trường địa chỉ
- **County**: Tên quận trong trường địa chỉ
- **State**: Tên bang trong trường địa chỉ
- **Zipcode**: Mã bưu chính trong trường địa chỉ
- **Country**: Tên quốc gia trong trường địa chỉ
- **Timezone**: Múi giờ dựa trên vị trí xảy ra tai nạn (miền đông, miền trung,...)
- **Airport_Code**: Trạm thời tiết dựa trên sân bay, gần nhất với vị trí của vụ tai nạn
- **Weather_Timestamp**: Thời gian ghi nhận thông tin thời tiết (theo giờ địa phương)
- **Temperature(F)**: Hiển thị nhiệt độ (tính bằng Fahrenheit)
- **Wind_Chill(F)**: Hiển thị mức độ lạnh của không khí (tính bằng Fahrenheit)
- **Humidity(%)**: Hiển thị độ ẩm (tính bằng %)
- **Pressure(in)**: Hiển thị áp suất không khí (tính bằng inches)
- **Visibility(mi)**: Hiển thị tầm nhìn (tính bằng miles)
- **Wind_Direction**: Hiển thị hướng gió
- **Wind_Speed(mph)**: Hiển thị tốc độ gió (tính bằng miles / giờ)
- **Precipitation(in)**: Hiển thị lượng mưa tính theo inches (nếu có)
- **Weather_Condition**: Hiển thị tình trạng thời tiết (mưa, có tuyết, có sấm sét,...)
- **Amenity**: Chú thích địa điểm cho biết sự hiện diện của tiện nghi ở vị trí lân cận
- **Bump**: Chú thích địa điểm cho biết sự hiện diện của gờ giảm tốc ở vị trí lân cận
- **Crossing**: Chú thích địa điểm cho biết sự hiện diện của điểm băng qua ở vị trí lân cận
- **Give_Way**: Chú thích địa điểm cho biết sự hiện diện của đoạn nhường đường ở vị trí lân cận
- **Junction**: Chú thích địa điểm cho biết sự hiện diện của giao lộ ở vị trí lân cận
- **No_Exit**: Chú thích địa điểm cho biết sự hiện diện của đoạn đường không có lối ra ở vị trí lân cận
- **Railway**: Chú thích địa điểm cho biết sự hiện diện của đường sắt ở vị trí lân cận
- **Roundabout**: Chú thích địa điểm cho biết sự hiện diện của bùng binh ở vị trí lân cận
- **Station**: Chú thích địa điểm cho biết sự hiện diện của trạm tàu ở vị trí lân cận
- **Stop**: Chú thích địa điểm cho biết sự hiện diện của điểm dừng ở vị trí lân cận
- **Traffic_Calming**: Chú thích địa điểm cho biết sự hiện diện của các biện pháp giảm tốc ở vị trí lân cận
- **Traffic_Signal**: Chú thích địa điểm cho biết sự hiện diện của đèn giao thông ở vị trí lân cận
- **Turning_Loop**: Chú thích địa điểm cho biết sự hiện diện của vòng xoay ở vị trí lân cận
- **Sunrise_Sunset**: Hiển thị khoảng thời gian trong ngày (dựa vào bình minh / hoàng hôn)
- **Civil_Twilight**: Hiển thị khoảng thời gian trong ngày (dựa vào choạng vạng dân sự)
- **Nautical_Twilight**: Hiển thị khoảng thời gian trong ngày (dựa vào choạng vạng hải lý)
- **Astronomical_Twilight**: Hiển thị khoảng thời gian trong ngày (dựa vào choạng vạng thiên văn)
